In [ ]:
# SKRYPT EKSPERYMENTALNY
# Uruchamia serię eksperymentów z różnymi parametrami algorytmu,
# zapisuje statystyki dla każdej kombinacji oraz mierzy pełny
# czas działania. Dane wynikowe trafiają do pliku CSV.
#
# Główne elementy:
#   - rozgrzanie kompilatora Numba (aby uniknąć narzutu JIT)
#   - wielokrotne uruchamianie algorytmu z tymi samymi parametrami
#   - multiprocessing (każde powtórzenie w osobnym procesie)
#   - wyznaczanie średnich kosztów i czasów
#   - zapisywanie najlepszej (minimalnej) trasy
#   - zapis wyników przy użyciu save_experiment_results()

%load_ext autoreload
%autoreload 2

import time
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count

from src.utils.tsp_loader import load_tsp_matrix
from src.algorithms.ihc_numba import solve_tsp
from src.utils.result_saver import save_experiment_results
from src.utils.run_single_repeat import run_single_repeat

# USTAWIENIA
TSP_FILES = ["Dane_TSP_76.xlsx", "Dane_TSP_76.xlsx", "Dane_TSP_76.xlsx"]

PARAM_GRID = {
    # "n_starts": [400],         # liczba restartów wspinaczki
    # "max_iter": [4000],        # liczba iteracji jednej wspinaczki
    # "stop_no_improve": [400],  # limit stagnacji
    # "neighborhood_type": ["two_opt"],
    "n_starts": [1600],         # liczba restartów wspinaczki
    "max_iter": [16000],        # liczba iteracji jednej wspinaczki
    "stop_no_improve": [1600],  # limit stagnacji
    "neighborhood_type": ["two_opt"],
}

REPEATS = 6   # liczba powtórzeń dla każdej kombinacji parametrów
results = []  # tablica na wyniki


# ROZGRZANIE NUMBA (KOMPILACJA JIT)
print("Rozgrzewanie Numba (kompilacja JIT)...")
D = load_tsp_matrix(TSP_FILES[0])
_ = solve_tsp(D, {"n_starts": 2})
print("Kompilacja zakończona.\n")


# GŁÓWNA PĘTLA
start_total = time.perf_counter()

for tsp_file in TSP_FILES:
    print(f"\nInstancja: {tsp_file}")
    D = load_tsp_matrix(tsp_file)

    for n_starts in PARAM_GRID["n_starts"]:
        for max_iter in PARAM_GRID["max_iter"]:
            for stop_no_improve in PARAM_GRID["stop_no_improve"]:
                for neighborhood_type in PARAM_GRID["neighborhood_type"]:

                    params = {
                        "n_starts": n_starts,
                        "max_iter": max_iter,
                        "stop_no_improve": stop_no_improve,
                        "neighborhood_type": neighborhood_type,
                        "use_delta": True
                    }

                    # multiprocessing — równoległe powtórzenia
                    with Pool(processes=cpu_count()) as pool:
                        parallel_jobs = [
                            (solve_tsp, D, params) for _ in range(REPEATS)
                        ]

                        results_parallel = pool.map(run_single_repeat, parallel_jobs)

                    costs = [c for c, _, _ in results_parallel]
                    routes = [r for _, r, _ in results_parallel]
                    runtimes = [t for _, _, t in results_parallel]

                    # wybór najlepszej trasy
                    min_cost = min(costs)
                    best_route_overall = routes[costs.index(min_cost)]
                    route_str = "-".join(map(str, best_route_overall))

                    # zapis danych
                    results.append({
                        "instance": tsp_file,
                        "n_starts": n_starts,
                        "max_iter": max_iter,
                        "stop_no_improve": stop_no_improve,
                        "neighborhood_type": neighborhood_type,
                        "mean_cost": round(np.mean(costs), 3),
                        "mean_runtime": np.mean(runtimes),
                        "min_cost": round(min_cost, 3),
                        "min_route": route_str,
                    })


# PODSUMOWANIE I ZAPIS WYNIKÓW
end_total = time.perf_counter()
elapsed = end_total - start_total

print(f"\nŁączny czas eksperymentów: {elapsed/60:.2f} min ({elapsed:.2f} sek)")

df = pd.DataFrame(results)
save_experiment_results(df, filename="with_delta__results.csv", time_seconds=int(elapsed), subfolder="IHC")


# RAPORT
print("\nPodsumowanie statystyczne (średnie po neighborhood_type):")
print(df.groupby("neighborhood_type")[["mean_cost", "min_cost", "mean_runtime"]]
      .mean().round(3))

best = df.loc[df["mean_cost"].idxmin()]
print(f"\nNajlepsze parametry:\n{best.to_dict()}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Rozgrzewanie Numba (kompilacja JIT)...
Kompilacja zakończona.


Instancja: Dane_TSP_76.xlsx

Łączny czas eksperymentów: 0.24 min (14.13 sek)

Podsumowanie (pierwsze 20 wierszy):
        instance  n_starts  max_iter  stop_no_improve neighborhood_type  mean_cost  mean_runtime  min_cost                                                                                                                                                                                                                    min_route
Dane_TSP_76.xlsx      1600     16000             1600           two_opt  98950.547     13.005557 98144.347 50-49-51-56-55-52-53-54-42-43-27-26-28-33-32-29-30-31-19-20-5-10-9-8-7-6-4-3-2-75-76-1-23-22-21-25-24-46-45-44-48-47-69-68-67-70-0-74-14-13-12-11-15-16-17-18-37-36-38-39-35-34-40-41-60-59-58-61-62-57-63-64-73-72-71-65-66

Podsumowanie statystyczne (średnie po neighborhood_type):
                   